# US Unemployment Rate by County, 1990-2016
Thanks to the US Department of Labor's Bureau of Labor Statistics

# 1) Importing libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 2) Reading Dataset

In [2]:
df=pd.read_csv("output.csv")

In [3]:
df.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [4]:
df.shape

(885548, 5)

# 3) Knowing data

In [5]:
df.dtypes.value_counts()

object     3
float64    1
int64      1
dtype: int64

In [6]:
# Checking all variables with their data-type
def _tbl_dtype(dataset):
    sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
    return sum_dtype

In [7]:
table_dtype = _tbl_dtype(df)
table_dtype

,Data Type
Year,int64
Rate,float64
Month,object
State,object
County,object


In [8]:
# Checking unique categorical variables
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

Month       12
State       47
County    1752
dtype: int64

In [9]:
df['Month'].value_counts()

March        74540
May          74100
November     74092
June         74018
July         73743
January      73718
August       73629
February     73611
October      73604
December     73590
April        73584
September    73319
Name: Month, dtype: int64

# 4) Using US minimum wage dataset

In [10]:
df2=pd.read_csv("Minimum Wage Data.csv", encoding="latin")

In [11]:
df2.head()

,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.10,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


In [12]:
df2.shape

(2750, 9)

In [13]:
gb = df2.groupby("State")
gb.get_group("Alabama").set_index("Year").head()

,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
Year,,,,,,,,
1968,Alabama,...,NaN,0.0,0.0,34.783333,0.0,0.0
1969,Alabama,...,NaN,0.0,0.0,36.683333,0.0,0.0
1970,Alabama,...,NaN,0.0,0.0,38.825000,0.0,0.0
1971,Alabama,...,NaN,0.0,0.0,40.491667,0.0,0.0
1972,Alabama,...,NaN,0.0,0.0,41.816667,0.0,0.0


In [14]:
act_min_wage = pd.DataFrame()

for name, group in df2.groupby("State"):
    if act_min_wage.empty:
        act_min_wage = group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name})
    else:
        act_min_wage = act_min_wage.join(group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name}))

act_min_wage.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),...,Tennessee,Texas,U.S. Virgin Islands,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1968,0.0,15.12,3.37,1.12,11.88,7.20,10.08,9.00,9.00,8.28,...,0.0,0.00,NaN,7.20,10.08,0.0,11.52,7.20,9.00,8.64
1969,0.0,14.33,3.19,1.07,11.26,6.83,9.56,8.53,8.53,7.85,...,0.0,0.00,NaN,6.83,9.56,0.0,10.92,6.83,8.53,8.19
1970,0.0,13.54,3.02,7.09,10.64,6.45,10.32,8.06,10.32,8.38,...,0.0,0.00,NaN,6.45,10.32,0.0,10.32,6.45,8.38,8.38
1971,0.0,12.99,2.89,6.80,10.20,6.18,9.89,7.73,9.89,8.04,...,0.0,0.00,NaN,6.18,9.89,0.0,9.89,6.18,8.04,8.04
1972,0.0,12.57,2.80,7.19,9.88,5.99,11.08,9.58,9.58,9.58,...,0.0,8.38,NaN,7.19,9.58,0.0,9.58,7.19,8.68,8.98


### Removing missing values

In [15]:
import numpy as np

act_min_wage = act_min_wage.replace(0, np.NaN).dropna(axis=1)
act_min_wage.head()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),Guam,Hawaii,...,Pennsylvania,Puerto Rico,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1968,15.12,1.12,11.88,7.20,10.08,9.00,9.00,8.28,9.00,9.00,...,8.28,3.10,10.08,3.06,7.20,10.08,11.52,7.20,9.00,8.64
1969,14.33,1.07,11.26,6.83,9.56,8.53,8.53,7.85,8.53,8.53,...,7.85,2.94,9.56,2.90,6.83,9.56,10.92,6.83,8.53,8.19
1970,13.54,7.09,10.64,6.45,10.32,8.06,10.32,8.38,10.32,10.32,...,8.38,2.77,10.32,6.45,6.45,10.32,10.32,6.45,8.38,8.38
1971,12.99,6.80,10.20,6.18,9.89,7.73,9.89,8.04,9.89,9.89,...,8.04,2.66,9.89,6.18,6.18,9.89,9.89,6.18,8.04,8.04
1972,12.57,7.19,9.88,5.99,11.08,9.58,9.58,9.58,11.38,9.58,...,9.58,3.89,9.58,5.99,7.19,9.58,9.58,7.19,8.68,8.98


In [16]:
def get_min_wage(year, state):
    try:
        return act_min_wage.loc[year][state]
    except:
        return np.NaN

In [17]:
get_min_wage(2012, "Colorado")

8.33

### Combine features of two datasets

In [18]:
df['min_wage'] = list(map(get_min_wage, df['Year'], df['State'])) # notice list conversion for map

In [19]:
df.head()

,Year,Month,State,County,Rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


**Nothing much shown here**

In [20]:
df.tail()

,Year,Month,State,County,Rate,min_wage
885543,2009,November,Maine,Somerset County,10.5,8.46
885544,2009,November,Maine,Oxford County,10.5,8.46
885545,2009,November,Maine,Knox County,7.5,8.46
885546,2009,November,Maine,Piscataquis County,11.3,8.46
885547,2009,November,Maine,Aroostook County,9.0,8.46


In [21]:
# Rename rate to unemp. rate
df=df.rename(columns={"Rate": "Unemp_rate"})

In [22]:
df.columns

Index(['Year', 'Month', 'State', 'County', 'Unemp_rate', 'min_wage'], dtype='object')

In [23]:
df[['Unemp_rate','min_wage']].corr()

,Unemp_rate,min_wage
Unemp_rate,1.000000,0.153047
min_wage,0.153047,1.000000


In [24]:
df[['Unemp_rate','min_wage']].cov()

,Unemp_rate,min_wage
Unemp_rate,9.687873,0.651586
min_wage,0.651586,1.874228


# 5) Using US presidential results dataset

In [25]:
pres16 = pd.read_csv("pres16results.csv")
pres16.head(15)

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump
5,NaN,US,Darrell Castle,US,0.9951,180877.0,127592176.0,0.001418,Donald Trump
6,NaN,US,Gloria La Riva,US,0.9951,48308.0,127592176.0,0.000379,Donald Trump
7,NaN,US,Rocky De La Fuente,US,0.9951,32120.0,127592176.0,0.000252,Donald Trump
8,NaN,US,None of these candidates,US,0.9951,28824.0,127592176.0,0.000226,Donald Trump
9,NaN,US,Richard Duncan,US,0.9951,23501.0,127592176.0,0.000184,Donald Trump


In [26]:
pres16['cand'].value_counts()

Gary Johnson                 3165
Donald Trump                 3165
Hillary Clinton              3165
Jill Stein                   2648
Darrell Castle               1331
Rocky De La Fuente           1013
Evan McMullin                 806
Alyson Kennedy                407
Gloria La Riva                401
Jim Hedges                    225
Dan Vacek                     189
Lynn Kahn                     177
Mike Smith                    162
Emidio Soltysik               150
Chris Keniston                131
Tom Hoefling                  131
Laurence Kotlikoff            131
Monica Moorehead              126
Richard Duncan                 90
Rod Silva                      66
Ryan Scott                     66
Joseph Maldonado               66
Jerry White                    66
Frank Atwood                   66
Kyle Kopitke                   66
Michael Maturen                66
Princess Jacob                 66
Bradford Lyttle                66
Peter Skewes                   48
Scott Copeland

In [27]:
pres16.dtypes.value_counts()

object     5
float64    4
dtype: int64

In [28]:
# Checking all variables with their data-type
def _tbl_dtype(dataset):
    sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
    return sum_dtype

In [29]:
table_dtype = _tbl_dtype(pres16)
table_dtype

,Data Type
pct_report,float64
votes,float64
total_votes,float64
pct,float64
county,object
fips,object
cand,object
st,object
lead,object


In [30]:
# Checking unique categorical variables
pres16.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

county    1969
fips      3289
cand        32
st          52
lead         2
dtype: int64

In [31]:
top_candidates = pres16.head(10)['cand'].values
print(top_candidates)

['Donald Trump' 'Hillary Clinton' 'Gary Johnson' 'Jill Stein'
 'Evan McMullin' 'Darrell Castle' 'Gloria La Riva' 'Rocky De La Fuente'
 ' None of these candidates' 'Richard Duncan']


In [32]:
county_2015 = df.copy()[(df['Year']==2015) & (df['Month']=="February")]

In [33]:
county_2015.head()

,Year,Month,State,County,Unemp_rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


In [34]:
pres16['st'].unique()

array(['US', 'CA', 'FL', 'TX', 'NY', 'PA', 'IL', 'OH', 'MI', 'NC', 'GA',
       'VA', 'NJ', 'MA', 'WI', 'MN', 'MO', 'WA', 'IN', 'CO', 'TN', 'MD',
       'SC', 'AL', 'AZ', 'LA', 'KY', 'OR', 'CT', 'IA', 'OK', 'MS', 'KS',
       'NV', 'AR', 'UT', 'NE', 'NM', 'ME', 'NH', 'WV', 'ID', 'MT', 'RI',
       'DE', 'HI', 'SD', 'ND', 'VT', 'DC', 'WY', 'AK', nan], dtype=object)

# 6) Adding post codes

In [40]:
# We also have it saved from previous work i.e US minimum wage kaggle challenge file
state_abbv2 = pd.read_csv("state_abbv.csv", index_col=0)
state_abbv2.head()

,Postal Code
State/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [41]:
state_abbv_dict = state_abbv2.to_dict()['Postal Code']

In [42]:
state_abbv_dict

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY

In [43]:
county_2015['State'] = county_2015['State'].map(state_abbv_dict)

In [44]:
county_2015.tail()

,Year,Month,State,County,Unemp_rate,min_wage
2797,2015,February,ME,Somerset County,8.4,7.92
2798,2015,February,ME,Oxford County,6.8,7.92
2799,2015,February,ME,Knox County,6.1,7.92
2800,2015,February,ME,Piscataquis County,7.0,7.92
2801,2015,February,ME,Aroostook County,7.2,7.92


In [45]:
print(len(county_2015))
print(len(pres16))

2802
18475


**We want to merge these dataset. There is one problem. The feature names i.e columns are different. State in country_2015 and st in pres16. We need to have them same for merging or joining**

In [46]:
pres16.rename(columns={"county": "County", "st": "State"}, inplace=True)
pres16.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [47]:
# Setting multiple indecies i.e County and State
for dataset in [county_2015, pres16]:
    dataset.set_index(["County", "State"], inplace=True)

In [48]:
pres16.head()

fips             cand  pct_report       votes  total_votes  \
County State                                                              
NaN    US      US     Donald Trump      0.9951  60350241.0  127592176.0   
       US      US  Hillary Clinton      0.9951  60981118.0  127592176.0   
       US      US     Gary Johnson      0.9951   4164589.0  127592176.0   
       US      US       Jill Stein      0.9951   1255968.0  127592176.0   
       US      US    Evan McMullin      0.9951    451636.0  127592176.0   

                   pct          lead  
County State                          
NaN    US     0.472993  Donald Trump  
       US     0.477938  Donald Trump  
       US     0.032640  Donald Trump  
       US     0.009844  Donald Trump  
       US     0.003540  Donald Trump

In [50]:
county_2015.head()

,,Year,Month,Unemp_rate,min_wage
County,State,,,,
Newton County,MS,2015,February,6.1,NaN
Panola County,MS,2015,February,9.4,NaN
Monroe County,MS,2015,February,7.9,NaN
Hinds County,MS,2015,February,6.1,NaN
Kemper County,MS,2015,February,10.6,NaN


We have two index now.  

In [53]:
pres16 = pres16[pres16['cand']=="Donald Trump"]
pres16 = pres16[['pct']]
pres16.dropna(inplace=True)
pres16.head(2)

pct
County State          
NaN    US     0.472993
       CA     0.330641

In [54]:
county_2015.head(2)

,,Year,Month,Unemp_rate,min_wage
County,State,,,,
Newton County,MS,2015,February,6.1,NaN
Panola County,MS,2015,February,9.4,NaN


In [55]:
all_together = county_2015.merge(pres16, on=["County", "State"])
all_together.dropna(inplace=True)
all_together.head()

,,Year,Month,Unemp_rate,min_wage,pct
County,State,,,,,
Major County,OK,2015,February,2.6,2.11,0.864960
Pottawatomie County,OK,2015,February,4.5,2.11,0.701342
Johnston County,OK,2015,February,6.5,2.11,0.770057
Jefferson County,OK,2015,February,5.0,2.11,0.812367
Beaver County,OK,2015,February,2.8,2.11,0.888243


In [56]:
all_together.drop("Year", axis=1, inplace=True)
all_together.head()

,,Month,Unemp_rate,min_wage,pct
County,State,,,,
Major County,OK,February,2.6,2.11,0.864960
Pottawatomie County,OK,February,4.5,2.11,0.701342
Johnston County,OK,February,6.5,2.11,0.770057
Jefferson County,OK,February,5.0,2.11,0.812367
Beaver County,OK,February,2.8,2.11,0.888243


In [57]:
all_together.corr()

,Unemp_rate,min_wage,pct
Unemp_rate,1.000000,0.186689,-0.085985
min_wage,0.186689,1.000000,-0.325036
pct,-0.085985,-0.325036,1.000000


**Higher wages have a negative correlation with percentage vote . So, those who have lower wages seem to have voted for Trump OR higher the wage, less likely to vote for Trump<br>
Same pattern is seen in umployment**

In [58]:
all_together.cov()

,Unemp_rate,min_wage,pct
Unemp_rate,5.743199,0.683870,-0.031771
min_wage,0.683870,2.336451,-0.076602
pct,-0.031771,-0.076602,0.023772
